In [8]:
import pandas as pd

In [10]:
#Подключение к БД и вывод содержимого таблицы 'flights'

In [12]:
import psycopg2
engine=psycopg2.connect("dbname='demo' user='postgres' host='localhost' port='5432' password='123'")
df=pd.read_sql('select * from flights', con=engine)
df

C:\Users\#slash\AppData\Local\Temp\ipykernel_9036\4009591350.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql('select * from flights', con=engine)


,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,182,PG0402,2017-09-01 09:25:00+00:00,2017-09-01 10:20:00+00:00,DME,LED,Scheduled,321,NaT,NaT
1,1996,PG0335,2017-08-26 06:30:00+00:00,2017-08-26 08:35:00+00:00,DME,JOK,Scheduled,CN1,NaT,NaT
2,5979,PG0384,2017-08-26 09:10:00+00:00,2017-08-26 09:40:00+00:00,DME,BZK,Scheduled,SU9,NaT,NaT
3,8136,PG0138,2017-08-28 07:15:00+00:00,2017-08-28 08:20:00+00:00,VKO,RTW,Scheduled,CR2,NaT,NaT
4,10455,PG0277,2017-09-12 08:45:00+00:00,2017-09-12 12:10:00+00:00,SVO,OVB,Scheduled,773,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...
65659,65660,PG0063,2017-06-26 16:25:00+00:00,2017-06-26 17:10:00+00:00,SKX,SVO,Arrived,CR2,2017-06-26 16:27:00+00:00,2017-06-26 17:12:00+00:00
65660,65661,PG0063,2017-08-10 16:25:00+00:00,2017-08-10 17:10:00+00:00,SKX,SVO,Arrived,CR2,2017-08-10 16:27:00+00:00,2017-08-10 17:13:00+00:00
65661,65662,PG0063,2017-06-16 16:25:00+00:00,2017-06-16 17:10:00+00:00,SKX,SVO,Arrived,CR2,2017-06-16 16:28:00+00:00,2017-06-16 17:14:00+00:00
65662,65663,PG0063,2017-09-10 16:25:00+00:00,2017-09-10 17:10:00+00:00,SKX,SVO,Scheduled,CR2,NaT,NaT


In [16]:
import numpy as n

In [18]:
#Запрос к БД. Вывод информации по загрузке рейсов

In [20]:
import psycopg2
query= f"""
with departure_trips AS (SELECT flights.departure_airport as airport, to_char(flights.actual_departure,'YYYY-MM-DD')
as airdate, count(boarding_passes.ticket_no) as amount
FROM flights
inner join boarding_passes
on (flights.flight_id=boarding_passes.flight_id)
where flights.status='Arrived'
group by flights.departure_airport, flights.actual_departure
order by flights.actual_departure),
arrival_trips AS (SELECT flights.arrival_airport as airport, to_char(flights.actual_arrival,'YYYY-MM-DD') as airdate, count(boarding_passes.ticket_no) as amount
FROM flights
inner join boarding_passes
on (flights.flight_id=boarding_passes.flight_id)
where flights.status='Arrived'
group by flights.arrival_airport, flights.actual_arrival
order by flights.actual_arrival)
select airdate, airport, airports_data.airport_name -> 'ru' as airname, sum(amount),
case when sum(amount) < 100 then 'Малая нагрузка'
     when sum(amount) > 1000 then 'Большая нагрузка'
else 'Средняя нагрузка'
end as airport_analys
from
(select arrival_trips.airport, arrival_trips.airdate, arrival_trips.amount from arrival_trips
union 
select departure_trips.airport, departure_trips.airdate, departure_trips.amount from departure_trips)
inner join airports_data
on airports_data.airport_code=airport
group by airdate, airport, airports_data.airport_name
order by airdate, airport;"""
engine=psycopg2.connect("dbname='demo' user='postgres' host='localhost' port='5432' password='123'")
df=pd.read_sql(query, con=engine)

C:\Users\#slash\AppData\Local\Temp\ipykernel_9036\3898673718.py:32: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df=pd.read_sql(query, con=engine)


In [22]:
#Вывод первых 15-ти строк

In [24]:
df.head(15)

,airdate,airport,airname,sum,airport_analys
0,2017-05-17,AAQ,Витязево,53.0,Малая нагрузка
1,2017-05-17,ABA,Абакан,11.0,Малая нагрузка
2,2017-05-17,AER,Сочи,134.0,Средняя нагрузка
3,2017-05-17,ARH,Талаги,36.0,Малая нагрузка
4,2017-05-17,ASF,Астрахань,33.0,Малая нагрузка
5,2017-05-17,BAX,Барнаул,8.0,Малая нагрузка
6,2017-05-17,BQS,Игнатьево,10.0,Малая нагрузка
7,2017-05-17,BTK,Братск,23.0,Малая нагрузка
8,2017-05-17,BZK,Брянск,241.0,Средняя нагрузка
9,2017-05-17,CEK,Челябинск,67.0,Малая нагрузка


In [26]:
# вывод типов данных

In [28]:
type (df)

pandas.core.frame.DataFrame

In [30]:
df.dtypes

airdate            object
airport            object
airname            object
sum               float64
airport_analys     object
dtype: object

In [ ]:
#число уникальных значений по столбцу "model"

In [32]:
b=df['airname'].value_counts()
print(b)

airname
Омск-Центральный       92
Ульяновск-Восточный    92
Уфа                    92
Богашёво               92
Краснодар              92
                       ..
Усинск                 37
Магадан                37
Стрежевой              26
Горно-Алтайск          25
Нягань                 25
Name: count, Length: 94, dtype: int64


In [ ]:
#число повторяющихся значений по столбцам

In [34]:
df1=df.apply(lambda x:sum(x.duplicated()))
print(df1)

airdate           8007
airport           8005
airname           8005
sum               6673
airport_analys    8096
dtype: int64


In [40]:
#минимальное значение по столбцу 'sum'

In [42]:
value_min=df['sum'].min()
print('Минимальное количество людей на аэропорт за день:', value_min)

минимальное кол-во мест в самолете: 1.0


In [ ]:
#максимальное значение количества мест в самолете(столбец 'total_seats')

In [44]:
value_max=df['sum'].max()
print('Максимальное количество людей на аэропорт за день:', value_max)

Максимальное количество людей на аэропорт за день: 5868.0


In [46]:
#подсчет медианы по загрузке аэропортов(столбец 'sum')

In [48]:
med=df['sum'].median()
print(med)

181.0


In [50]:
#подсчет среднего значения загрузки аэропортов (столбец 'sum')

In [52]:
sred=df['sum'].mean()
sred


426.73700456846524